In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 이미지 추가 전처리
def apply_gaussian_filter(image):
    np_image = image.numpy()  # 텐서를 넘파이 배열로 변환
    if len(np_image.shape) == 3:
        np_image = np_image.transpose(1, 2, 0)  # 차원 변경 (C, H, W) -> (H, W, C)
        filtered_image = cv2.GaussianBlur(np_image, (3, 3), 0)
        filtered_image = filtered_image.transpose(2, 0, 1)  # 차원 변경 (H, W, C) -> (C, H, W)
        return torch.from_numpy(filtered_image)  # 넘파이 배열을 텐서로 변환
    else:
        return image

In [3]:
# 클래스명
class_names = ['Atelectasis', 'Cardiomegaly', 'Edema',
               'Effusion', 'Fibrosis', 'Normal',
               'Pneumonia', 'Pneumothorax', 'Tuberculosis']

# 데이터 로딩 및 전처리
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=224),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: apply_gaussian_filter(x)), # 가우시안 필터링 적용
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
# 모델 정의 및 불러오기
model = torch.load('MultiClassification.h5')
model.to(device)
model.eval()

# 이미지 불러오기
image = Image.open('../data/NIH_sampled/test/Mass/Normal/00000624_005.png')
image = image.convert('RGB')
input_image = transform(image).unsqueeze(0).to(device)

# 예측 수행
with torch.no_grad():
    output = model(input_image)

# 예측 결과 확인
_, predicted_idx = torch.max(output, 1)
predicted_label = class_names[predicted_idx.item()]
print(f"Predicted Label : {predicted_label}")

Predicted Label : Normal
